# Authorisation of medical Staff

This notebook tests granting and retracting authorisations for different levels of medical actor, and assigning medical staff a permission level. This version does not yet include lab authorisation levels and actions.

In [1]:
% Set up required to reset environment
?- cd('~/work'), ['dec'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

We initially expect that a diagnosis will have been made by a medical staff member authorised to do so. 

Current medical staff are represented by **medStaff(Actor,Level)**, where **Actor** is the staff member's ID and **Level** is their permission level. Staff can be hired by the **newStaff** event and let go by the **oldStaff** event.

Staff members have a permission level i.e. **nurse** or **doctor**. Different levels are able to perform different actions. A level can have actions granted or revoked, and a staff member can have their permission level changed. In this scenario, each staff member may only have one permission level, however different permission levels may share some authorisations. 

In [2]:
% File: labInitiates.pl
initiates(diagnosis(RID,PatID,Condition,Result,Actor), diagnosed(RID,PatID,Condition,Result,Actor),_).
initially(exp_rule(happ(diagnosis(_,_,_,_,Actor)), individualAuthorised(diagnosis,Actor))).

% Initiate / terminate employment.
initiates(newStaff(Actor,Level), medStaff(Actor)=Level,_).
terminates(oldStaff(Actor), medStaff(Actor)=_,_).
% Grant new staff members appropriate action authorisations for their level.
initiates(newStaff(Actor,Level), individualAuthorised(Action,Actor),T):- holdsAt(levelAuthority(Level,Action),T).
% Revoke action authorisations from old staff members.
terminates(oldStaff(Actor), individualAuthorised(Action,Actor),T).

% Are staff authorised at time zero?
initially(individualAuthorised(Action,Actor)):- initially(levelAuthority(Level,Action)), initially(medStaff(Actor)=Level).


% Add or revoke authorities for levels. 
initiates(authorityAdd(Level, Action), levelAuthority(Level,Action),T).
terminates(authorityRemove(Level, Action), levelAuthority(Level, Action),T).
% Add or revoke level authorities to medical staff
initiates(authorityAdd(Level, Action), individualAuthorised(Action, Actor),T):- holdsAt(medStaff(Actor)=Level,T).
terminates(authorityRemove(Level,Action), individualAuthorised(Action,Actor),T):- holdsAt(medStaff(Actor)=Level,T).

% Demotions or Promotions
% terminates(promotion(Actor,NewLevel), medStaff(Actor)=OldLevel,T).
initiates(promotion(Actor,NewLevel), medStaff(Actor)=NewLevel,T).
initiates(promotion(Actor, NewLevel), individualAuthorised(Action,Actor),T):- holdsAt(levelAuthority(NewLevel,Action),T).
terminates(promotion(Actor, NewLevel), individualAuthorised(Action,Actor),T):- holdsAt(levelAuthority(OldLevel,OldAction),T), \+ holdsAt(authority(NewLevel,OldAction),T).

We declare some starting authorisation levels and staff members. For example, nurses may request tests but may not may make diagnoses. **AH** and **EH** are staff members who are nurses. **EH** is promoted to **doctor** at time period 2. (Note that the promotion event can be used for promotion or demotion, currently there is not an explicit hierarchy of levels.)

In [3]:
% File: agents.pl
% Initial actions that different levels of staff may perform.
initially(levelAuthority(nurse,testRequests)).
initially(levelAuthority(doctor, testRequests)).
initially(levelAuthority(doctor, diagnosis)).
initially(levelAuthority(specialist, testRequests)).
initially(levelAuthority(specialist, diagnosis)).
initially(levelAuthority(specialist, brainDeath)).


% Some starting staff members.
initially(medStaff('AH')=nurse).
initially(medStaff('EH')=nurse).
initially(medStaff('PS')=doctor).

% A specialist joins
happensAtNarrative(newStaff('JW',specialist),0).
% Nurses are allowed to perform additional actions after time period 1.
happensAtNarrative(authorityAdd(nurse, nurseAct1),1).
happensAtNarrative(authorityAdd(nurse, nurseAct2),1).
% 'EH', who was a nurse, is promoted to a doctor. 
happensAtNarrative(promotion('EH',doctor),2).
% An acceptor is hired and the specialist is let go.
happensAtNarrative(newStaff('GM',acceptor),2).
happensAtNarrative(oldStaff('JW'),2).
% Changes to actions performed by acceptors or nurses.
happensAtNarrative(authorityAdd(acceptor, donorDec),3).
happensAtNarrative(authorityRemove(nurse, nurseAct2),4).

% EH makes a diagnosis decision (test for exp_rule)
happensAtNarrative(diagnosis(1,123,'HIV',positive,'EH'),4).
% AH makes a diagnosis decision (test for exp_rule)
happensAtNarrative(diagnosis(1,123,'HIV',positive,'AH'),4).

At time zero, we can see that all three active medical staff may make test requests, while **PS** can also make diagnoses, as they are a doctor. **JW** joins the staff at this time.

In [4]:
?- tick(0).
?- holdsAt(F,0).
?- happensAt(E,0).

true.
F = individualAuthorised(testRequests, AH) ;
F = individualAuthorised(testRequests, EH) ;
F = individualAuthorised(testRequests, PS) ;
F = individualAuthorised(diagnosis, PS) ;
F = {'medStaff(AH)': 'nurse'} ;
F = {'medStaff(EH)': 'nurse'} ;
F = {'medStaff(PS)': 'doctor'} ;
F = exp_rule(happ(diagnosis(_3038, _3040, _3042, _3044, _3046)), individualAuthorised(diagnosis, _3046)) ;
F = levelAuthority(nurse, testRequests) ;
F = levelAuthority(doctor, testRequests) ;
F = levelAuthority(doctor, diagnosis) ;
F = levelAuthority(specialist, testRequests) ;
F = levelAuthority(specialist, diagnosis) ;
F = levelAuthority(specialist, brainDeath) .
E = newStaff(JW, specialist) .

At time one, **JW** is a member of staff and is able to undertake actions authorised for specialists. Nurses are granted additional authority at this time.

In [5]:
?- tick(1).
?- holdsAt(medStaff('JW')=Level,1).
?- holdsAt(individualAuthorised(Action,'JW'),1).
?- happensAt(E,1).


true.
Level = specialist .
Action = testRequests ;
Action = diagnosis ;
Action = brainDeath .
E = authorityAdd(nurse, nurseAct1) ;
E = authorityAdd(nurse, nurseAct2) .

At time 2, the two nurses have been granted the additional authorisations. At this time, **EH** is promoted to **doctor**, **GM** joins the staff and **JW** leaves.

In [6]:
?- tick(2).
?- holdsAt(individualAuthorised(X,Y),2), (Y='AH'; Y='EH').
?- happensAt(O,2).


true.
X = testRequests, Y = AH ;
X = testRequests, Y = EH ;
X = nurseAct1, Y = AH ;
X = nurseAct1, Y = EH ;
X = nurseAct2, Y = AH ;
X = nurseAct2, Y = EH .
O = promotion(EH, doctor) ;
O = newStaff(GM, acceptor) ;
O = oldStaff(JW) .

At time 3, **JW** is no longer a member of staff, but **GM** is. **GM** is currently not authorised to perform any action. At this time point, acceptors are granted authority to make donor decisions. **EH** is now a doctor, and can only perform actions authorised to doctors. They are no longer able to perform actions that were previously authorised due to their status as a **nurse**.

In [7]:
?- tick(3).
?- holdsAt(medStaff(ID)=Level,3).
?- holdsAt(individualAuthorised(Action,Actor),3).
?- happensAt(D,3).

true.
ID = AH, Level = nurse ;
ID = PS, Level = doctor ;
ID = EH, Level = doctor ;
ID = GM, Level = acceptor .
Action = testRequests, Actor = AH ;
Action = testRequests, Actor = PS ;
Action = diagnosis, Actor = PS ;
Action = nurseAct1, Actor = AH ;
Action = nurseAct2, Actor = AH ;
Action = testRequests, Actor = EH ;
Action = diagnosis, Actor = EH .
D = authorityAdd(acceptor, donorDec) .

At time 4, diagnoses are made by **AH** and **EH**. As **EH** is a doctor, the expectation that a diagnosis is made by someone authorised to do so is fulfilled by this action, but also separately violated by the diagnosis made by **AH**, who is not authorised to do so. 

At time 4, the authority for **nurses** to perform action **nurseAct2** is revoked. At time 5, **AH**, who is a nurse, no longer has the authority to perform this action. At time 5, **GM** has the authority to make donor decisions, as an **acceptor**.

In [8]:
?- initialiseDEC.
?- run(6).
?- happensAt(B,4).
?- holdsAt(levelAuthority(nurse,NurseAction),5).
?- holdsAt(individualAuthorised(AHAction,'AH'),5).
?- holdsAt(individualAuthorised(GMAction,'GM'),5).
?- holdsAt(exp(A,B,C,D),4).

true.
true.
B = authorityRemove(nurse, nurseAct2) ;
B = diagnosis(1, 123, HIV, positive, EH) ;
B = diagnosis(1, 123, HIV, positive, AH) ;
B = fulf(happ(diagnosis(1, 123, HIV, positive, EH)), individualAuthorised(diagnosis, EH), 4, individualAuthorised(diagnosis, EH)) ;
B = viol(happ(diagnosis(1, 123, HIV, positive, AH)), individualAuthorised(diagnosis, AH)) .
NurseAction = testRequests ;
NurseAction = nurseAct1 .
AHAction = testRequests ;
AHAction = nurseAct1 .
GMAction = donorDec .
A = happ(diagnosis(1, 123, HIV, positive, EH)), B = individualAuthorised(diagnosis, EH), C = 4, D = individualAuthorised(diagnosis, EH) .

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>